In [27]:
import pandas as pd
import numpy as np
import os, csv
from sklearn import preprocessing, cross_validation

In [28]:
trainQ = "Q11999"
testQ = "Q21999"

In [24]:
def load_df(quarter1,quarter2):  
    foldername= 'data_part2'
    historical_path= str(os.getcwd())+"\\"+ foldername
    train_path= historical_path+"\historical_data1_time_"+str(trainQ)+".txt"
    test_path= historical_path+"\historical_data1_time_"+str(testQ)+".txt"
    global train_df
    global test_df
    column_names=['loan_seq_number', 'month', 'current_actual_upb', 'delq_status','loan_age', 'rem_months', 'repurchase_flag', 'modification_flag', 'zero_balance_code',
                      'zero_bal_date', 'current_int_rate', 'current_def_upb', 'ddlpi', 'mi_recoveries', 'net_sales_proceeds',
                      'non_mi_recoveries', 'expenses', 'legal_costs', 'maint_pres_costs', 'taxes_ins', 'misc_expenses',
                      'actual_loss_calc', 'modification_cost']
    
    train_df = pd.read_table(os.path.join(train_path), 
                        delimiter='|', names=column_names, index_col=None, nrows=1048576, usecols=list(np.arange(23)))
    test_df = pd.read_table(os.path.join(test_path), 
                        delimiter='|', names=column_names, index_col=None, nrows=1048576, usecols=list(np.arange(23)))
   
    
load_df(trainQ, testQ)

    
def cleandf(df):
    df.delq_status = df.delq_status.replace('R', '1').astype('float64')
    df.rem_months = df.rem_months.replace(np.nan, 0)
    df.rem_months = df.rem_months.astype('category')
    df.repurchase_flag = df.repurchase_flag.replace(np.nan, 0)
    df.repurchase_flag = df.repurchase_flag.astype('category')
    df.modification_flag = df.modification_flag.replace(np.nan, 0)
    df.modification_flag = df.modification_flag.astype('category')
    df.zero_balance_code = df.zero_balance_code.replace(np.nan, 0)
    df.zero_balance_code = df.zero_balance_code.astype('category')
    df.zero_bal_date = df.zero_bal_date.replace(np.nan, 0)
    df.zero_bal_date = df.zero_bal_date.astype('category')
    df.current_def_upb = df.current_def_upb.replace(np.nan, 0)
    df.current_def_upb = df.current_def_upb.astype('category')
    df.ddlpi = df.ddlpi.replace(np.nan, 0)
    df.ddlpi = df.ddlpi.astype('category')
    df.mi_recoveries = df.mi_recoveries.replace(np.nan, 0)
    df.net_sales_proceeds = df.net_sales_proceeds.replace(np.nan, 0)
    df.net_sales_proceeds = df.net_sales_proceeds.replace('C', 1)
    df.net_sales_proceeds = df.net_sales_proceeds.replace('U', 0)
    df.net_sales_proceeds.astype('float64')
    df.non_mi_recoveries = df.non_mi_recoveries.replace(np.nan, 0)
    df.expenses = df.expenses.replace(np.nan, 0)
    df.legal_costs = df.legal_costs.replace(np.nan, 0)
    df.maint_pres_costs = df.maint_pres_costs.replace(np.nan, 0)
    df.taxes_ins = df.taxes_ins.replace(np.nan, 0)
    df.misc_expenses = df.misc_expenses.replace(np.nan, 0)
    df.actual_loss_calc = df.actual_loss_calc.replace(np.nan, 0)
    df.modification_cost = df.modification_cost.replace(np.nan, 0)

cleandf(train_df)
cleandf(test_df)

C:\Users\smits_000\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (7,14) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\Users\smits_000\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (3,7,14) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [25]:
def f(row):
    if row['delq_status'] > 0:
        val = 1
    else:
        val = 0
    return val
    
#Create dummy variables
train_df_dummies = pd.get_dummies(train_df[['repurchase_flag', 'modification_flag']])
test_df_dummies = pd.get_dummies(test_df[['repurchase_flag', 'modification_flag']])

dropped_train_df = train_df.drop(['loan_seq_number','repurchase_flag', 'modification_flag'],axis=1)
dropped_test_df = test_df.drop(['loan_seq_number','repurchase_flag', 'modification_flag'],axis=1)

global final_train_df
global final_test_df
final_train_df = pd.concat([dropped_train_df, train_df_dummies], axis=1)
final_test_df = pd.concat([dropped_test_df, test_df_dummies], axis=1)

#create target variable 
final_train_df['Deliquent'] = final_train_df.apply(f, axis=1)
final_test_df['Deliquent'] = final_test_df.apply(f, axis=1)
    

In [26]:
train_CSV = os.getcwd() + '\\train.csv'
final_train_df.to_csv(train_CSV,index= False, encoding='utf-8')
test_CSV = os.getcwd() + '\\test.csv'
final_test_df.to_csv(test_CSV,index= False, encoding='utf-8')